In [1]:
import numpy as np
import pandas as pd
import requests # 웹 상의 데이터를 가져올 때
from bs4 import BeautifulSoup # 가져온 데이터에서 필요한 정보를 골라낼(파싱할) 때

# 확인

In [2]:
# 셀레니움 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [3]:
# 크롬창 열기
driver = webdriver.Chrome('./chromedriver.exe')

In [4]:
# 설정한 url로 데이터 get함수로 요청
# 검색지역 지정
keyword = "고성군"

# url정의
url = f'https://www.mangoplate.com/search/{keyword}?keyword={keyword}&page=1'
driver.get(url)

In [6]:
# 첫 번째 게시물 클릭
driver.find_element_by_xpath('/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a/div/img').click()

In [7]:
# 무분별한 크롤링을 막기위한 방지코드가 존재
# 아래 사이트에서 User-Agent 정보를 복사하여 header를 추가하여 request한다.
# www.useragentstring.com/
# headers = {'User-Agent': '본인의 User-Agent'} # 각자 본인의 User-Agent 정보 입력
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'} # 각자 본인의 User-Agent 정보 입력 

# requsts로 url에 정보요청 (.get 함수를 자주 사용하면 서비스 제공자가 블락할 수 있어 1번 사용한 후, 주석처리를 하고 변수로 설정하여 사용한다.)
resp = requests.get(url, headers = headers)

# 정보를 html 변환 (보기 쉽게)
soup = BeautifulSoup(resp.text, 'lxml')

In [8]:
# 보고 싶은 정보만 선별
title = soup.select("h2.title")[0].text
search_point = soup.select("strong.point.search_point")[0].text
etc = soup.select("p.etc")[0].text
view_count = soup.select("span.view_count")[0].text
review_count = soup.select("span.review_count")[0].text

In [9]:
# 원하는 항목의 데이터만 추출 
address = soup.select('img')[0]['alt']

In [10]:
soup.select('img')[0]['alt']

'백촌막국수 사진 - 강원도 고성군 토성면 백촌리 162'

In [11]:
title, search_point, etc, view_count, review_count, address

('백촌막국수\n',
 '4.3',
 '강원 고성군 - 국수 / 면 요리',
 '54,320',
 '82',
 '백촌막국수 사진 - 강원도 고성군 토성면 백촌리 162')

In [ ]:
# 첫 번째 사진
/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[1]/figure/a/div/img

In [ ]:
# 두 번째 사진
/html/body/main/article/div[2]/div/div/section/div[3]/ul/li[1]/div[2]/figure/a/div/img

# 실습

In [12]:
# 빈 딕셔너리 생성
dic_ = {
    'title': [],
    'search_point': [],
    'etc': [],
    'view_count': [],
    'review_count': [],
    'address' : []
}

# 검색지역 지정
keyword = "고성군"

# 1~21 페이지 설정
# 숨김페이지 21페이지
for page in range(21):
    # url정의
    url = f'https://www.mangoplate.com/search/{keyword}?keyword={keyword}&page={page}'

    # 무분별한 크롤링을 막기위한 방지코드가 존재
    # 아래 사이트에서 User-Agent 정보를 복사하여 header를 추가하여 request한다.
    # www.useragentstring.com/
    # headers = {'User-Agent': '본인의 User-Agent'} # 각자 본인의 User-Agent 정보 입력
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'} # 각자 본인의 User-Agent 정보 입력 
    
    # requsts로 url에 정보요청 (.get 함수를 자주 사용하면 서비스 제공자가 블락할 수 있어 1번 사용한 후, 주석처리를 하고 변수로 설정하여 사용한다.)
    resp = requests.get(url, headers = headers)

    # 정보를 html 변환 (보기 쉽게)
    soup = BeautifulSoup(resp.text, 'lxml')

    # 20개의 게시물 크롤링
    for i in range(20):
        title = soup.select("h2.title")[i].text
        search_point = soup.select("strong.point.search_point")[i].text
        etc = soup.select("p.etc")[i].text
        view_count = soup.select("span.view_count")[i].text
        review_count = soup.select("span.review_count")[i].text
        address = soup.select('img')[i]['alt']
        
        # address에서 'title'과 "사진 -" 문자열 제거(문제)
        # address에서 강원도 -> 원도로 되는 문제 발생
        # .lstrip(title + " 사진")으로 "-"는 남겨둔다.
        address = address.lstrip(title + " 사진")
        address = address.replace("- ", "")

        dic_['title'].append(title)
        dic_['search_point'].append(search_point)
        dic_['etc'].append(etc)
        dic_['view_count'].append(view_count)
        dic_['review_count'].append(review_count)
        dic_['address'].append(address)

In [13]:
# 'address' 필요없는 문자열 제거
title, address

('올라\n', '강원도 고성군 토성면 천진리 110')

In [14]:
address.lstrip(title + " 사진 - ")

'강원도 고성군 토성면 천진리 110'

In [15]:
# dic_['title']의 "\n" 과 "" 처리 필요
dic_['title']

['백촌막국수\n',
 '동해반점\n',
 '온더버튼\n',
 '장미경양식\n',
 '바다정원\n',
 '에이프레임\n',
 '해피홀리\n',
 '금화정막국수\n',
 '만성횟집\n',
 '영순네횟집\n',
 '스퀘어루트\n',
 '소울브릿지\n',
 '글라스하우스\n',
 '게섯거라\n',
 '테일\n',
 '오미냉면\n',
 '커피고\n',
 '태시트\n',
 '스위밍 터틀\n',
 '나폴리커피하우스\n',
 '백촌막국수\n',
 '동해반점\n',
 '온더버튼\n',
 '장미경양식\n',
 '바다정원\n',
 '에이프레임\n',
 '해피홀리\n',
 '금화정막국수\n',
 '만성횟집\n',
 '영순네횟집\n',
 '스퀘어루트\n',
 '소울브릿지\n',
 '글라스하우스\n',
 '게섯거라\n',
 '테일\n',
 '오미냉면\n',
 '커피고\n',
 '태시트\n',
 '스위밍 터틀\n',
 '나폴리커피하우스\n',
 '화진포박포수가든\n',
 '헬로우씨카페\n',
 '본래순대\n                                    (고성군청점)\n',
 '본토대가\n',
 '백도막국수\n',
 '남경식당\n',
 '고깃집\n',
 '이스트사이드바이브클럽\n',
 '동루골막국수\n',
 '드레\n',
 '항구 백두산횟집\n',
 '남파랑가\n',
 '화진포메밀막국수\n',
 '도자기별\n',
 '석이네회센터\n',
 '동명막국수\n',
 '메밀촌봉평막국수\n',
 '회뜨는마을\n',
 '광범이네활어센터\n',
 '고성빵가\n',
 '진미분식\n',
 '제비호식당\n                                    (본점)\n',
 '소가야횟집\n',
 '산북소나무막국수\n',
 '송지호횟집\n',
 '더착한삼겹살\n',
 '성문집\n',
 '수성반점\n',
 '구이가\n                                    (경동대점)\n',
 '수앤커피하우스\n',
 '고향막국수\n',
 '오앤씨\n',
 '부부횟집\n  

In [17]:
df = pd.DataFrame(dic_)
df

,title,search_point,etc,view_count,review_count,address
0,백촌막국수\n,4.3,강원 고성군 - 국수 / 면 요리,"54,320",82,강원도 고성군 토성면 백촌리 162
1,동해반점\n,4.0,강원 고성군 - 정통 중식 / 일반 중식,"6,553",10,강원도 고성군 현내면 초도리 225
2,온더버튼\n,3.9,강원 고성군 - 카페 / 디저트,"7,200",21,강원도 고성군 죽왕면 문암진리 134-26
3,장미경양식\n,3.9,강원 고성군 - 기타 한식,"15,865",10,강원도 고성군 거진읍 거진리 277-7
4,바다정원\n,3.8,강원 고성군 - 카페 / 디저트,"10,876",46,강원도 고성군 토성면 용촌리 75-36
...,...,...,...,...,...,...
415,메트로폴리스\n,,강원 고성군 - 카페 / 디저트,119,0,강원도 고성군 죽왕면 문암진리 81-25
416,BBQ치킨\n (고성...,,강원 고성군 - 닭 / 오리 요리,117,0,강원도 고성군 현내면 대진리 111-7
417,끼리네한상\n,,경남 고성군 - 해산물 요리,113,0,경상남도 고성군 고성읍 서외리 1-6
418,일차돌\n (고성점)\n,,강원 고성군 - 고기 요리,104,0,강원도 고성군 간성읍 신안리 248-2


In [86]:
pd.set_option('display.max_rows', None) ## 모든 행을 출력한다.
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.

In [18]:
# 줄 바꿈기호(\n) 삭제
df['title'] = df["title"].replace("\n", "", regex=True)

In [19]:
# 확인
df["title"]

0                                                 백촌막국수
1                                                  동해반점
2                                                  온더버튼
3                                                 장미경양식
4                                                  바다정원
                             ...                       
415                                              메트로폴리스
416    BBQ치킨                                    (고성대진점)
417                                               끼리네한상
418        일차돌                                    (고성점)
419                                                  올라
Name: title, Length: 420, dtype: object

In [20]:
# 빈칸 제거
df['title'] = df["title"].replace(" ", "", regex=True)

In [21]:
df

,title,search_point,etc,view_count,review_count,address
0,백촌막국수,4.3,강원 고성군 - 국수 / 면 요리,"54,320",82,강원도 고성군 토성면 백촌리 162
1,동해반점,4.0,강원 고성군 - 정통 중식 / 일반 중식,"6,553",10,강원도 고성군 현내면 초도리 225
2,온더버튼,3.9,강원 고성군 - 카페 / 디저트,"7,200",21,강원도 고성군 죽왕면 문암진리 134-26
3,장미경양식,3.9,강원 고성군 - 기타 한식,"15,865",10,강원도 고성군 거진읍 거진리 277-7
4,바다정원,3.8,강원 고성군 - 카페 / 디저트,"10,876",46,강원도 고성군 토성면 용촌리 75-36
...,...,...,...,...,...,...
415,메트로폴리스,,강원 고성군 - 카페 / 디저트,119,0,강원도 고성군 죽왕면 문암진리 81-25
416,BBQ치킨(고성대진점),,강원 고성군 - 닭 / 오리 요리,117,0,강원도 고성군 현내면 대진리 111-7
417,끼리네한상,,경남 고성군 - 해산물 요리,113,0,경상남도 고성군 고성읍 서외리 1-6
418,일차돌(고성점),,강원 고성군 - 고기 요리,104,0,강원도 고성군 간성읍 신안리 248-2


In [22]:
# 경상남도 고성군이 들어간 경우가 있어 제거
df = df[df["etc"].str.contains("강원 고성군")]

In [23]:
# 중복 row 제거
df.drop_duplicates(ignore_index = True, inplace=True)

<ipython-input-23-53c86a37f1ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(ignore_index = True, inplace=True)


In [24]:
df

,title,search_point,etc,view_count,review_count,address
0,백촌막국수,4.3,강원 고성군 - 국수 / 면 요리,"54,320",82,강원도 고성군 토성면 백촌리 162
1,동해반점,4.0,강원 고성군 - 정통 중식 / 일반 중식,"6,553",10,강원도 고성군 현내면 초도리 225
2,온더버튼,3.9,강원 고성군 - 카페 / 디저트,"7,200",21,강원도 고성군 죽왕면 문암진리 134-26
3,장미경양식,3.9,강원 고성군 - 기타 한식,"15,865",10,강원도 고성군 거진읍 거진리 277-7
4,바다정원,3.8,강원 고성군 - 카페 / 디저트,"10,876",46,강원도 고성군 토성면 용촌리 75-36
...,...,...,...,...,...,...
277,해바라기식당,,강원 고성군 - 해산물 요리,119,0,강원도 고성군 거진읍 거진리 29-3
278,메트로폴리스,,강원 고성군 - 카페 / 디저트,119,0,강원도 고성군 죽왕면 문암진리 81-25
279,BBQ치킨(고성대진점),,강원 고성군 - 닭 / 오리 요리,117,0,강원도 고성군 현내면 대진리 111-7
280,일차돌(고성점),,강원 고성군 - 고기 요리,104,0,강원도 고성군 간성읍 신안리 248-2


In [25]:
# serch_point = "" -> 0 으로 변환
df["search_point"].replace("", 0.0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [26]:
df

,title,search_point,etc,view_count,review_count,address
0,백촌막국수,4.3,강원 고성군 - 국수 / 면 요리,"54,320",82,강원도 고성군 토성면 백촌리 162
1,동해반점,4.0,강원 고성군 - 정통 중식 / 일반 중식,"6,553",10,강원도 고성군 현내면 초도리 225
2,온더버튼,3.9,강원 고성군 - 카페 / 디저트,"7,200",21,강원도 고성군 죽왕면 문암진리 134-26
3,장미경양식,3.9,강원 고성군 - 기타 한식,"15,865",10,강원도 고성군 거진읍 거진리 277-7
4,바다정원,3.8,강원 고성군 - 카페 / 디저트,"10,876",46,강원도 고성군 토성면 용촌리 75-36
...,...,...,...,...,...,...
277,해바라기식당,0.0,강원 고성군 - 해산물 요리,119,0,강원도 고성군 거진읍 거진리 29-3
278,메트로폴리스,0.0,강원 고성군 - 카페 / 디저트,119,0,강원도 고성군 죽왕면 문암진리 81-25
279,BBQ치킨(고성대진점),0.0,강원 고성군 - 닭 / 오리 요리,117,0,강원도 고성군 현내면 대진리 111-7
280,일차돌(고성점),0.0,강원 고성군 - 고기 요리,104,0,강원도 고성군 간성읍 신안리 248-2


In [27]:
# 저장
df.to_csv("고성_맛집.csv", encoding='cp949', index=False)